In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
import pickle

In [4]:
df = pd.read_csv('Churn_Modelling.csv')
df.head()
df.columns

Index(['RowNumber', 'CustomerId', 'Surname', 'CreditScore', 'Geography',
       'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'Exited'],
      dtype='object')

In [5]:
## drop the columns
df = df.drop(['RowNumber', 'CustomerId', 'Surname'],axis=1)

In [6]:
## encode the categorical variable
label_encoder = LabelEncoder()
df['Gender'] = label_encoder.fit_transform(df['Gender'])

In [7]:
## One-hot encoder
#create encoder
onehotencoder = OneHotEncoder(sparse_output=False)

# fit and transform
encoded = onehotencoder.fit_transform(df[['Geography']])

## get feature names
feature_names = onehotencoder.get_feature_names_out(['Geography'])

## create data frame with encoded columns
encoded_df = pd.DataFrame(encoded, columns=feature_names, index=df.index)

## drop original column and concatenate encoded columns
df = df.drop('Geography',axis=1)
df = pd.concat([df, encoded_df], axis=1)

df.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [8]:
## independent features
X = df.drop('EstimatedSalary', axis=1)
## dependent feature
y = df['EstimatedSalary']

In [9]:
## train test split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [10]:
## scaling
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [11]:
## save the pickle files
with open('encoder.pkl', 'wb') as file:
    pickle.dump(label_encoder, file)

with open('scaler.pkl','wb') as file:
    pickle.dump(onehotencoder,file)

with open('scaler.pkl','wb') as file:
    pickle.dump(scaler, file)

In [15]:
## train ANN

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

## build the model
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1)
]
)

## compile the model
model.compile(optimizer='adam',loss='mean_absolute_error',metrics=['mae'])

d:\End-to-end-Projects\6-DL-project\venv\Lib\site-packages\keras\src\layers\core\dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [16]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_6 (Dense)                 │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [17]:
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

## setup the tensorboard
log_dir = 'regressionlogs/fit/'+ datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq = 1)

In [18]:
## setup warly stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [19]:
## train the model
history = model.fit(
    X_train, y_train,
    validation_data = (X_test, y_test),
    epochs=100,
    callbacks=[early_stopping, tensorboard_callback]
)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 18s 26ms/step - loss: 100386.7344 - mae: 100386.7344 - val_loss: 98548.7344 - val_mae: 98548.7344
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 6s 16ms/step - loss: 99723.3594 - mae: 99723.3594 - val_loss: 97195.0547 - val_mae: 97195.0547
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 6s 17ms/step - loss: 97330.6172 - mae: 97330.6172 - val_loss: 93677.1094 - val_mae: 93677.1094
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 5s 16ms/step - loss: 92574.7188 - mae: 92574.7188 - val_loss: 87753.7109 - val_mae: 87753.7109
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 9s 37ms/step - loss: 85515.2812 - mae: 85515.2812 - val_loss: 79883.7422 - val_mae: 79883.7422
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 10s 34ms/step - loss: 76997.8672 - mae: 76997.8672 - val_loss: 71371.3359 - val_mae: 71371.3359
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 10s 31ms/step - loss: 68395.8516 - mae: 68395.8516 - val_loss: 63553.4961 - val_mae: 63553.4961
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 12

In [20]:
%load_ext tensorboard

In [21]:
%tensorboard --logdir regressionlogs/fit

ERROR: Timed out waiting for TensorBoard to start. It may still be running as pid 12364.

In [23]:
## save the model

model.save('model_reg.keras')